In [1]:
import numpy as np
import tensorflow as tf
import librosa

import matplotlib.pyplot as plt

2022-12-04 08:23:09.903254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 08:23:10.246393: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-04 08:23:10.246418: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-04 08:23:11.236692: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
y, sr = librosa.load("../data/raw/maestro-v3.0.0/2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_01_R1_2018_wav--1.wav", sr=44100)

In [3]:
frame_size = 128
y_pad = np.pad(y, (0, frame_size - len(y) % frame_size), mode="constant")
tf.signal.frame(y, frame_size, frame_size, pad_end=True)

2022-12-04 08:23:25.174407: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-04 08:23:25.174839: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-04 08:23:25.174882: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-MDDIFBJE): /proc/driver/nvidia/version does not exist
2022-12-04 08:23:25.177814: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(113846, 128), dtype=float32, numpy=
array([[-0.00111721, -0.00125049, -0.00114745, ..., -0.00208398,
        -0.00215918, -0.00221768],
       [-0.0021319 , -0.00213825, -0.00224164, ..., -0.00401902,
        -0.0040878 , -0.00408183],
       [-0.00411096, -0.00413494, -0.00407899, ..., -0.00401888,
        -0.00404358, -0.00400979],
       ...,
       [ 0.00359923,  0.00362357,  0.00357396, ...,  0.00346783,
         0.00348935,  0.0034698 ],
       [ 0.0034917 ,  0.00343648,  0.00337807, ...,  0.00260985,
         0.00268212,  0.0026379 ],
       [ 0.0026536 ,  0.00263106,  0.00255113, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)>

In [4]:
y

array([-0.00111721, -0.00125049, -0.00114745, ...,  0.00127404,
        0.00128858,  0.        ], dtype=float32)

In [5]:
sample = np.arange(5)
frames = tf.signal.frame(sample, 2, 2, pad_end=True)
frames.shape[0]

3

In [130]:
from dataclasses import dataclass

import note_seq
import tensorflow as tf

from typing import Iterable, List, Tuple
from note_seq import NoteSequence


@dataclass
class Note:
    pitch: int
    velocity: int


def make_frames(audio_signal: Iterable[float], sample_rate: int, frame_length: int, frame_step: int, pad_end=True, pad_value=0) -> Tuple[tf.Tensor, List[float]]:
    frames = tf.signal.frame(audio_signal, frame_length, frame_step, pad_end, pad_value)
    frame_time = frame_length / sample_rate
    times = [frame_time * i for i in range(frames.shape[0])]
    return frames, times


def tokenize(ns: NoteSequence, times: Iterable[float], frame_time: float, single_note=True) -> List[Note]:
    ns_sorted = sorted(ns.notes, key=lambda note: note.start_time)
    ns_iter = 0
    notes = []

    curr_note = None
    prev_notes = []
    for time in times:
        while ns_iter < len(ns_sorted) - 1 and ns_sorted[ns_iter].start_time < time:
            prev_notes.append(ns_sorted[ns_iter])
            ns_iter += 1
        curr_note = ns_sorted[ns_iter]

        notes.append([])

        if curr_note.start_time - time < frame_time:
            notes[-1].append(Note(curr_note.pitch, curr_note.velocity))

        for note in prev_notes:
            if note is not None and note.end_time > time:
                notes[-1].append(Note(note.pitch, note.velocity))

        if single_note and len(notes[-1]) > 1:
            notes[-1] = [notes[-1][0]]
        

    assert len(notes) == len(times)
    return notes

def detokenize(notes: Iterable[Iterable[Note]], times: Iterable[float], frame_time: float) -> NoteSequence:
    ns = NoteSequence()
    for notes_inner, time in zip(notes, times):
        for note in notes_inner:
            ns.notes.append(
                NoteSequence.Note(
                    pitch=note.pitch,
                    velocity=note.velocity,
                    start_time=time,
                    end_time=time + frame_time
                )
            )

    return ns


In [124]:
frames, times = make_frames(sample, 1, 2, 2)
frames, times

(<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
 array([[0, 1],
        [2, 3],
        [4, 0]])>,
 [0.0, 2.0, 4.0])

In [125]:
test_tokenize = note_seq.midi_file_to_note_sequence("../data/raw/tokenize_test.MID")

test_tokenize.notes[0].end_time = 0.3
note_seq.plot_sequence(test_tokenize)

Loading BokehJS ...

In [126]:
times = [0, 0.2, 0.4, 0.6, 0.8]
notes = tokenize(test_tokenize, times, 0.2)
notes
detokenized = detokenize(notes, times, 0.2)
note_seq.plot_sequence(detokenized)

Loading BokehJS ...

In [127]:
test_tokenize = note_seq.midi_file_to_note_sequence("../data/raw/tokenize_test_1.MID")

note_seq.plot_sequence(test_tokenize)

Loading BokehJS ...

In [128]:
times = [0.25 * i for i in range(8)]
notes = tokenize(test_tokenize, times, 0.25, True)
notes
detokenized = detokenize(notes, times, 0.25)
note_seq.plot_sequence(detokenized)

Loading BokehJS ...

In [21]:
test_notes = note_seq.midi_file_to_note_sequence("../data/raw/test_midi.MID")

note_seq.plot_sequence(test_notes)

Loading BokehJS ...

In [ ]:
times = [0, 4]

In [13]:
def trim_overlapping_notes(ns: NoteSequence):
    trimmed_ns = note_seq.NoteSequence()
    trimmed_ns.CopyFrom(ns)
    for i in range(len(trimmed_ns.notes) - 1):
        cur_note = trimmed_ns.notes[i]
        next_note = trimmed_ns.notes[i + 1]
        if trimmed_ns.notes[i].end_time > trimmed_ns.notes[i + 1].start_time:
            if trimmed_ns.notes[i].end_time < trimmed_ns.notes[i + 1].end_time:
                trimmed_ns.notes[i + 1].start_time = trimmed_ns.notes[i].end_time
            # trimmed_ns.notes[i].end_time = trimmed_ns.notes[i + 1].start_time
    return trimmed_ns

In [14]:
trimmed_ns = trim_overlapping_notes(test_notes)
note_seq.plot_sequence(trimmed_ns)

Loading BokehJS ...

In [15]:
splitted = note_seq.split_note_sequence(test_notes, 0.1)
concated = note_seq.concatenate_sequences(splitted, [])
note_seq.plot_sequence(test_notes)
note_seq.plot_sequence(concated)

Loading BokehJS ...

Loading BokehJS ...

In [ ]:
def split_note()